# Simplified TensorFlow Training and Deployment with SageMaker Script Mode

## TensorFlow script mode training and serving
TensorFlow Script Mode Training and Serving is a powerful feature that streamlines the process of training and deploying machine learning models using TensorFlow on Amazon SageMaker. This approach allows you to execute your TensorFlow training script in SageMaker with minimal modifications, making it effortless to harness the full potential of your models.

## Seamless Integration with SageMaker Python SDK

The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) serves as your trusty companion in this journey. It effortlessly handles the task of transferring your script to a SageMaker training instance, saving you valuable time and effort. Once on the training instance, SageMaker's native TensorFlow support comes into play, setting up essential training-related environment variables and expertly executing your training script.

## Flexible scripting Options

Script mode offers flexibility in your choice of scripting, supporting Python scripts, Python modules, or even shell scripts. In this illustrative example, we employ a Python script to train a robust classification model on the renowned [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Our aim is to demonstrate just how straightforward it is to train a SageMaker model using TensorFlow 2.0 scripts with the SageMaker Python SDK.

# Set up the environment
Let's start by setting up the environment:

## Imports

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(project_root)

from src import mnist_tf2
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

## Set-up

In [ ]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

# Accessing MNIST Training Data

In our pursuit of machine learning excellence, we have easy access to the coveted MNIST dataset, hosted conveniently in public Amazon S3 buckets. Specifically, you can find this treasure trove of data in the S3 bucket `sagemaker-sample-data-<REGION>`, nestled comfortably under the `tensorflow/mnist` prefix.

The MNIST dataset comprises four invaluable .npy files, each playing a crucial role in our training journey:

1. **train_data.npy**: This file holds the training data, a goldmine of hand-drawn digit images that will fuel the training of our model.

2. **eval_data.npy**: Here, we find the evaluation data, providing a separate set of digit images to assess the performance of our model with the utmost precision.

3. **train_labels.npy**: The training labels, stored in this file, serve as the guiding light for our model, helping it learn the intricate art of digit classification.

4. **eval_labels.npy**: Lastly, the evaluation labels contained herein allow us to validate the accuracy and effectiveness of our trained model.

These files are the building blocks of our machine learning endeavor, empowering us to unlock the secrets hidden within the MNIST dataset and embark on a transformative journey toward model mastery.

In [ ]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

In [ ]:
!aws s3 ls $training_data_uri/

# Constructing a Script for Distributed Training

In our quest to harness the full power of machine learning, we've crafted a training script that is both efficient and adaptable. This script serves as the backbone of our distributed training process, allowing us to make the most out of our resources and achieve optimal results.

## Leveraging TensorFlow's Example

Our training script draws inspiration from TensorFlow's official [CNN MNIST example](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). We've taken this exemplary foundation and customized it to seamlessly integrate with SageMaker's functionality.

## Handling `model_dir` Parameter

One key modification involves handling the `model_dir` parameter passed in by SageMaker. This parameter represents an S3 path, opening doors to efficient data sharing during distributed training and enabling checkpointing and model persistence. Our script takes full advantage of this capability.

## Simplifying Argument Parsing

To enhance user-friendliness, we've incorporated an argument-parsing function. This feature simplifies the process of handling training-related variables, making the script easy to configure and adapt to specific needs.

## Exporting the Trained Model

At the culmination of the training job, our script includes a crucial step: exporting the trained model. This export operation sends the model to the path stored in the environment variable `SM_MODEL_DIR`. This variable always points to `/opt/ml/model`, a critical location because SageMaker automatically uploads all model artifacts from this folder to S3 at the end of training.

### Full Script

For your convenience, here is the complete script, ready to fuel your machine learning journey:

In [ ]:
!wget https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/main/Section-15-Custom-models/mnist-tf2.py

In [ ]:
# TensorFlow 2.x script
!pygmentize 'mnist-tf2.py'

# Create a training job using the TensorFlow estimator
The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job. Let's call out a couple important parameters here:

`py_version` is set to `'py3'` to indicate that we are using script mode since legacy mode supports only Python 2. Though Python 2 will be deprecated soon, you can use script mode with Python 2 by setting py_version to `py2` and `script_mode` to True.

`distribution` is used to configure the distributed training setup. It's required only if you are doing distributed training either across a cluster of instances or across multiple GPUs. Here we are using parameter servers as the distributed training schema. SageMaker training jobs run on homogeneous clusters. To make parameter server more performant in the SageMaker setup, we run a parameter server on every instance in the cluster, so there is no need to specify the number of parameter servers to launch. Script mode also supports distributed training with [Horovod](https://github.com/horovod/horovod). You can find the full documentation on how to configure distributions [here](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training).

`instance_type` specify the EC2 instance used for training. You should right-size your training instance based on the size of your data, algorithm and tasks. Here we choose `ml.c5.xlarge`. You can also read more about [G4dn](https://aws.amazon.com/ec2/instance-types/g4/) instances, which feature NVIDIA T4 GPUs and custom Intel Cascade Lake CPUs, and are optimized for machine learning inference and small scale training. Read more on [available instance types and pricing](https://aws.amazon.com/sagemaker/pricing/).  

`use_spot_instances`(Optional): For further cost optimization, you can leverage [managed Amazon EC2 Spot instances](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html) by setting this parameter to `True`. Managed spot training can optimize the cost of training models up to 90% over on-demand instances. SageMaker manages the Spot interruptions on your behalf. You can specify which training jobs use spot instances and a stopping condition that specifies how long Amazon SageMaker waits for a job to run using Amazon EC2 Spot instances. Full documentation [here](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-python-sdk/managed_spot_training_tensorflow_estimator/managed_spot_training_tensorflow_estimator.html).  

You can also initiate an estimator to train with TensorFlow 2.1 script. The only things that you will need to change are the script name and `framework_version`

In [ ]:
mnist_estimator = TensorFlow(entry_point='mnist-tf2.py',
                             role=role,
                             instance_count=2,
                             instance_type='ml.m5.large',
                             framework_version='2.1.0',
                             py_version='py3',
                             distribution={'parameter_server': {'enabled': True}})

# Calling `fit`
To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. fit creates a default channel named 'training', which points to this S3 location. In the training script we can then access the training data from the location stored in SM_CHANNEL_TRAINING. fit accepts a couple other types of input as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the TensorFlow container executes mnist-tf2.py, passing hyperparameters and model_dir from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and model_dir defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:

`python mnist.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`

When training is complete, the training job will upload the saved model for TensorFlow serving.

Calling fit to train a model with TensorFlow 2.1 script.

In [ ]:
mnist_estimator.fit(training_data_uri)

# Deploy the trained model to an endpoint
The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint, because we trained with script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol. The [Using your own inference code](https://render.githubusercontent.com/view/ipynb?color_mode=auto&commit=a5c9a21e6ed70fd51ab5178f3a35461473f7b379&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6177732f616d617a6f6e2d736167656d616b65722d6578616d706c65732f613563396132316536656437306664353161623531373866336133353436313437336637623337392f736167656d616b65722d707974686f6e2d73646b2f74656e736f72666c6f775f7363726970745f6d6f64655f747261696e696e675f616e645f73657276696e672f74656e736f72666c6f775f7363726970745f6d6f64655f747261696e696e675f616e645f73657276696e672e6970796e62&nwo=aws%2Famazon-sagemaker-examples&path=sagemaker-python-sdk%2Ftensorflow_script_mode_training_and_serving%2Ftensorflow_script_mode_training_and_serving.ipynb&repository_id=107937815&repository_type=Repository) document explains how SageMaker runs inference containers.

In [ ]:
# cell 08
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Invoke the endpoint
Let's download the training data and use that as input for inference.

In [ ]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

The formats of the input and the output data correspond directly to the request and response formats of the Predict method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using a numpy array as input, which will be serialized into the simplified JSON format. In addtion, TensorFlow serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow serving SageMaker endpoint [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

In [ ]:
np.argmax(predictions['predictions'][0])

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

# Delete the endpoint
Let's delete the endpoint we just created to prevent incurring any extra costs and then [verify](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)

In [ ]:
predictor.delete_endpoint()